# Bidirectional GAN

Ref.: DONAHUE, Jeff; KRÄHENBÜHL, Philipp; DARRELL, Trevor.  
      Adversarial feature learning. arXiv preprint arXiv:1605.09782, 2016.  
      https://arxiv.org/abs/1605.09782

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers import concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

Using TensorFlow backend.


# Generator

In [3]:
def build_generator(latent_dim, img_shape):
    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    z = Input(shape=(latent_dim,))
    gen_img = model(z)

    return Model(z, gen_img)

# Discriminator

In [4]:
def build_discriminator(latent_dim, img_shape):

    z = Input(shape=(latent_dim, ))
    img = Input(shape=img_shape)
    d_in = concatenate([z, Flatten()(img)])

    model = Dense(1024)(d_in)
    model = LeakyReLU(alpha=0.2)(model)
    model = Dropout(0.5)(model)
    model = Dense(1024)(model)
    model = LeakyReLU(alpha=0.2)(model)
    model = Dropout(0.5)(model)
    model = Dense(1024)(model)
    model = LeakyReLU(alpha=0.2)(model)
    model = Dropout(0.5)(model)
    validity = Dense(1, activation="sigmoid")(model)

    return Model([z, img], validity)

In [5]:
# Encoder

In [6]:
def build_encoder(latent_dim, img_shape):
    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(latent_dim))

    model.summary()

    img = Input(shape=img_shape)
    z = model(img)

    return Model(img, z)

In [7]:
# auxiliary function

In [8]:
def do_sample_interval(G, latent_dim, epoch):
    r, c = 5, 5
    z = np.random.normal(size=(25, latent_dim))
    gen_imgs = G.predict(z)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [9]:
# train

In [10]:
def train(G, D, encoder, bigan_generator, 
          latent_dim, 
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):


        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Sample noise and generate img
        z = np.random.normal(size=(batch_size, latent_dim))
        imgs_ = G.predict(z)

        # Select a random batch of images and encode
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        z_ = encoder.predict(imgs)

        # Train the discriminator (img -> z is valid, z -> img is fake)
        d_loss_real = D.train_on_batch([z_, imgs], valid)
        d_loss_fake = D.train_on_batch([z, imgs_], fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator (z -> img is valid and img -> z is is invalid)
        g_loss = bigan_generator.train_on_batch([z, imgs], [valid, fake])

        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0  or epoch == epochs - 1:
            do_sample_interval(G, latent_dim, epoch)

# main()

In [11]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [12]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [13]:
# Build and compile the discriminator
D = build_discriminator(latent_dim, img_shape)
D.compile(loss=['binary_crossentropy'],
    optimizer=optimizer,
    metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
# Build the generator
G = build_generator(latent_dim, img_shape)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_7 (Dense)              (None, 784)              

In [15]:
# Build the encoder
encoder = build_encoder(latent_dim, img_shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dense_9 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 512)              

In [16]:
# The part of the bigan that trains the discriminator and encoder
D.trainable = False

In [17]:
# Generate image from sampled noise
z = Input(shape=(latent_dim, ))
img_ = G(z)

In [18]:
# Encode image
img = Input(shape=img_shape)
z_ = encoder(img)

In [19]:
# Latent -> img is fake, and img -> latent is valid
fake = D([z, img_])
valid = D([z_, img])

In [20]:
# Set up and compile the combined model
# Trains generator to fool the discriminator
bigan_generator = Model([z, img], [fake, valid])
bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
    optimizer=optimizer)

## train

In [21]:
epochs=4000
# epochs=40000
train(G, D, encoder, bigan_generator, latent_dim, epochs=epochs, batch_size=32, sample_interval=400)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.864189, acc: 42.19%] [G loss: 4.699399]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.485925, acc: 57.81%] [G loss: 4.574885]
2 [D loss: 0.347682, acc: 79.69%] [G loss: 5.124751]
3 [D loss: 0.187920, acc: 98.44%] [G loss: 4.948505]
4 [D loss: 0.125468, acc: 98.44%] [G loss: 6.786323]
5 [D loss: 0.093840, acc: 100.00%] [G loss: 7.774105]
6 [D loss: 0.054868, acc: 100.00%] [G loss: 8.652189]
7 [D loss: 0.046577, acc: 100.00%] [G loss: 9.138082]
8 [D loss: 0.033806, acc: 100.00%] [G loss: 10.126712]
9 [D loss: 0.023581, acc: 100.00%] [G loss: 10.825612]
10 [D loss: 0.017455, acc: 100.00%] [G loss: 11.089355]
11 [D loss: 0.013907, acc: 100.00%] [G loss: 10.904389]
12 [D loss: 0.009511, acc: 100.00%] [G loss: 11.177761]
13 [D loss: 0.010820, acc: 100.00%] [G loss: 11.022591]
14 [D loss: 0.009934, acc: 100.00%] [G loss: 11.748069]
15 [D loss: 0.009266, acc: 100.00%] [G loss: 12.428512]
16 [D loss: 0.006354, acc: 100.00%] [G loss: 12.914053]
17 [D loss: 0.007453, acc: 100.00%] [G loss: 12.741557]
18 [D loss: 0.005347, acc: 100.00%] [G loss: 13.673919]
19 [D loss: 

149 [D loss: 0.291632, acc: 81.25%] [G loss: 10.490429]
150 [D loss: 0.225071, acc: 89.06%] [G loss: 8.301505]
151 [D loss: 0.483443, acc: 85.94%] [G loss: 9.368088]
152 [D loss: 0.252907, acc: 89.06%] [G loss: 9.332811]
153 [D loss: 0.209827, acc: 92.19%] [G loss: 9.515306]
154 [D loss: 0.169592, acc: 92.19%] [G loss: 8.284096]
155 [D loss: 0.400339, acc: 82.81%] [G loss: 8.744150]
156 [D loss: 0.376499, acc: 84.38%] [G loss: 10.949204]
157 [D loss: 0.210445, acc: 87.50%] [G loss: 9.785616]
158 [D loss: 0.429634, acc: 79.69%] [G loss: 9.861772]
159 [D loss: 0.180323, acc: 90.62%] [G loss: 7.653506]
160 [D loss: 0.555147, acc: 75.00%] [G loss: 9.333048]
161 [D loss: 0.686792, acc: 70.31%] [G loss: 11.060529]
162 [D loss: 0.259673, acc: 82.81%] [G loss: 9.189779]
163 [D loss: 0.226137, acc: 93.75%] [G loss: 5.720976]
164 [D loss: 0.326528, acc: 84.38%] [G loss: 8.711897]
165 [D loss: 0.165148, acc: 92.19%] [G loss: 7.760811]
166 [D loss: 0.204426, acc: 90.62%] [G loss: 7.864395]
167 [D 

...

3265 [D loss: 0.525643, acc: 73.44%] [G loss: 3.361223]
3266 [D loss: 0.451677, acc: 78.12%] [G loss: 3.221721]
3267 [D loss: 0.479413, acc: 79.69%] [G loss: 3.283047]
3268 [D loss: 0.690313, acc: 64.06%] [G loss: 2.592065]
3269 [D loss: 0.700601, acc: 60.94%] [G loss: 2.953001]
3270 [D loss: 0.625700, acc: 75.00%] [G loss: 2.444839]
3271 [D loss: 0.719646, acc: 56.25%] [G loss: 2.555356]
3272 [D loss: 0.659890, acc: 65.62%] [G loss: 3.086574]
3273 [D loss: 0.533493, acc: 76.56%] [G loss: 3.232439]
3274 [D loss: 0.596252, acc: 64.06%] [G loss: 2.718241]
3275 [D loss: 0.435476, acc: 81.25%] [G loss: 2.894541]
3276 [D loss: 0.504477, acc: 71.88%] [G loss: 3.432902]
3277 [D loss: 0.601411, acc: 68.75%] [G loss: 2.855613]
3278 [D loss: 0.533804, acc: 76.56%] [G loss: 3.189409]
3279 [D loss: 0.504041, acc: 71.88%] [G loss: 2.915085]
3280 [D loss: 0.520907, acc: 71.88%] [G loss: 2.803129]
3281 [D loss: 0.649686, acc: 56.25%] [G loss: 2.756424]
3282 [D loss: 0.588966, acc: 68.75%] [G loss: 3.

3413 [D loss: 0.544128, acc: 75.00%] [G loss: 3.392192]
3414 [D loss: 0.460549, acc: 78.12%] [G loss: 3.036723]
3415 [D loss: 0.543616, acc: 70.31%] [G loss: 3.295111]
3416 [D loss: 0.549843, acc: 73.44%] [G loss: 2.843476]
3417 [D loss: 0.484451, acc: 81.25%] [G loss: 2.625529]
3418 [D loss: 0.602568, acc: 75.00%] [G loss: 2.979724]
3419 [D loss: 0.613075, acc: 70.31%] [G loss: 3.486600]
3420 [D loss: 0.559192, acc: 73.44%] [G loss: 3.224716]
3421 [D loss: 0.673946, acc: 60.94%] [G loss: 2.498665]
3422 [D loss: 0.609582, acc: 68.75%] [G loss: 2.867684]
3423 [D loss: 0.556514, acc: 70.31%] [G loss: 2.637017]
3424 [D loss: 0.548399, acc: 67.19%] [G loss: 2.569274]
3425 [D loss: 0.667736, acc: 60.94%] [G loss: 2.757951]
3426 [D loss: 0.525345, acc: 79.69%] [G loss: 3.282719]
3427 [D loss: 0.700873, acc: 59.38%] [G loss: 2.454931]
3428 [D loss: 0.618030, acc: 68.75%] [G loss: 2.465562]
3429 [D loss: 0.549596, acc: 71.88%] [G loss: 2.766921]
3430 [D loss: 0.724005, acc: 59.38%] [G loss: 2.

3560 [D loss: 0.540055, acc: 71.88%] [G loss: 3.097000]
3561 [D loss: 0.620266, acc: 62.50%] [G loss: 2.658696]
3562 [D loss: 0.485779, acc: 73.44%] [G loss: 2.833357]
3563 [D loss: 0.659470, acc: 65.62%] [G loss: 2.774822]
3564 [D loss: 0.704767, acc: 57.81%] [G loss: 2.392279]
3565 [D loss: 0.510133, acc: 76.56%] [G loss: 2.693129]
3566 [D loss: 0.781602, acc: 57.81%] [G loss: 3.019930]
3567 [D loss: 0.523778, acc: 78.12%] [G loss: 3.080822]
3568 [D loss: 0.515726, acc: 75.00%] [G loss: 3.197758]
3569 [D loss: 0.651636, acc: 59.38%] [G loss: 2.635407]
3570 [D loss: 0.572993, acc: 68.75%] [G loss: 3.012463]
3571 [D loss: 0.675864, acc: 65.62%] [G loss: 2.149776]
3572 [D loss: 0.611007, acc: 73.44%] [G loss: 2.697643]
3573 [D loss: 0.528739, acc: 71.88%] [G loss: 2.689749]
3574 [D loss: 0.591732, acc: 64.06%] [G loss: 3.021060]
3575 [D loss: 0.643059, acc: 59.38%] [G loss: 2.731271]
3576 [D loss: 0.668105, acc: 62.50%] [G loss: 2.965577]
3577 [D loss: 0.587908, acc: 65.62%] [G loss: 2.

3707 [D loss: 0.540963, acc: 71.88%] [G loss: 2.548944]
3708 [D loss: 0.580330, acc: 68.75%] [G loss: 2.544756]
3709 [D loss: 0.570832, acc: 67.19%] [G loss: 2.378438]
3710 [D loss: 0.736408, acc: 54.69%] [G loss: 2.807677]
3711 [D loss: 0.603473, acc: 67.19%] [G loss: 2.926075]
3712 [D loss: 0.570650, acc: 67.19%] [G loss: 2.701061]
3713 [D loss: 0.666046, acc: 59.38%] [G loss: 2.648087]
3714 [D loss: 0.471713, acc: 81.25%] [G loss: 2.499906]
3715 [D loss: 0.655482, acc: 62.50%] [G loss: 2.459013]
3716 [D loss: 0.483661, acc: 76.56%] [G loss: 3.158827]
3717 [D loss: 0.581458, acc: 65.62%] [G loss: 3.054222]
3718 [D loss: 0.492288, acc: 78.12%] [G loss: 3.085001]
3719 [D loss: 0.587320, acc: 68.75%] [G loss: 2.404190]
3720 [D loss: 0.618951, acc: 60.94%] [G loss: 3.033170]
3721 [D loss: 0.586456, acc: 73.44%] [G loss: 2.593637]
3722 [D loss: 0.626749, acc: 64.06%] [G loss: 2.682327]
3723 [D loss: 0.599684, acc: 67.19%] [G loss: 2.801939]
3724 [D loss: 0.592460, acc: 67.19%] [G loss: 2.

3854 [D loss: 0.546722, acc: 70.31%] [G loss: 2.437437]
3855 [D loss: 0.644024, acc: 64.06%] [G loss: 2.721611]
3856 [D loss: 0.679270, acc: 64.06%] [G loss: 2.658547]
3857 [D loss: 0.639845, acc: 65.62%] [G loss: 2.826665]
3858 [D loss: 0.572661, acc: 70.31%] [G loss: 2.873740]
3859 [D loss: 0.558512, acc: 68.75%] [G loss: 2.534676]
3860 [D loss: 0.649670, acc: 68.75%] [G loss: 2.911393]
3861 [D loss: 0.649183, acc: 68.75%] [G loss: 2.284823]
3862 [D loss: 0.691107, acc: 56.25%] [G loss: 3.015588]
3863 [D loss: 0.590393, acc: 70.31%] [G loss: 2.444590]
3864 [D loss: 0.625578, acc: 64.06%] [G loss: 2.632223]
3865 [D loss: 0.631359, acc: 70.31%] [G loss: 2.769390]
3866 [D loss: 0.548049, acc: 76.56%] [G loss: 2.890488]
3867 [D loss: 0.511631, acc: 70.31%] [G loss: 2.823705]
3868 [D loss: 0.681166, acc: 60.94%] [G loss: 2.561763]
3869 [D loss: 0.576370, acc: 70.31%] [G loss: 2.875548]
3870 [D loss: 0.595910, acc: 68.75%] [G loss: 3.058036]
3871 [D loss: 0.538600, acc: 76.56%] [G loss: 2.